# Week 4

Haoyang Han<br>


several method of model comparison and performance metric comparison:

**performance metric comparison**

---

since this problem is classification(and somehow biased), here we will discuss following metrics:

1. Confusion metric
2. Accuracy
3. Precision
4. Recall
5. F1 Score

This comparison should based on logistic regression.



---





    



In [5]:
# !pip install brewer2mpl
# import necessary package

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

pd.set_option('display.max_row',100)
pd.set_option('display.max_column',100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}


plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

# Version
print(mpl.__version__)  #> 3.0.0
print(sns.__version__)  #> 0.9.0

3.0.2
0.9.0


###  1. import data and description

In [6]:
!ls

Data Cleaning.ipynb    model_comparison.ipynb
labeled_data.csv       transformed_data.csv


In [14]:
data = pd.read_csv('transformed_data.csv')

In [8]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,6 days 16 hours 36 mins
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,1 month and 3 days
H2O cluster name:,H2O_from_python_kevin_w2g0mh
H2O cluster total nodes:,1
H2O cluster free memory:,3.958 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [9]:
h2o.shutdown(prompt = True)

[WARNING] in <ipython-input-9-1ea49f0103fa> line 1:
    >>> h2o.shutdown(prompt = True)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
Are you sure you want to shutdown the H2O instance running at http://localhost:54321 (Y/N)? N


In [15]:
data.head()

,Unnamed: 0,Lat,Alt,Provider,FixSatCount,HasRadialAccuracy,HasVerticalAccuracy,HasSpeed,Speed,HasBearing,Bearing,BearingAccuracy,n,Attack
0,0,-0.01498,1.07356,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-1.68701,False
1,1,-0.01498,1.07356,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-1.68700,False
2,2,-0.01498,1.07356,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-1.68700,False
3,3,-0.01498,1.07356,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-1.68699,False
4,4,-0.01498,1.07356,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-1.68698,False


In [16]:
data.describe()

,Unnamed: 0,Lat,Alt,Provider,FixSatCount,HasRadialAccuracy,HasVerticalAccuracy,HasSpeed,Speed,HasBearing,Bearing,BearingAccuracy,n
count,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000,368537.00000
mean,184268.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,0.00000,-0.00000,-0.00000,-0.00000,0.00000,0.00000
std,106387.61242,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
min,0.00000,-2.06798,-15.19010,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,-0.51352,-1.69095
25%,92134.00000,-0.62328,-0.55987,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,-0.51352,-0.75771
50%,184268.00000,-0.01829,0.00380,-0.92435,-0.83310,-0.92435,-0.92435,-0.92265,-0.33750,-0.42627,-0.22922,0.00000,-0.11425
75%,276402.00000,0.00325,0.76296,1.08183,0.99372,1.08183,1.08183,1.08383,-0.05012,-0.42627,-0.22922,0.00000,0.80479
max,368536.00000,2.14346,4.60951,1.08183,2.51607,1.08183,1.08183,1.08383,17.55184,2.34591,55.57270,57.08746,2.04687


In [20]:
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=500, shuffle = True)

/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


### 2. metric comparison

In [24]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 500)

lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
y_pred = lr.predict(X_test)

In [ ]:
'''
metrics.accuracy_score(y_true, y_pred[, …])	Accuracy classification score.
metrics.auc(x, y[, reorder])	Compute Area Under the Curve (AUC) using the trapezoidal rule
metrics.average_precision_score(y_true, y_score)	Compute average precision (AP) from prediction scores
metrics.balanced_accuracy_score(y_true, y_pred)	Compute the balanced accuracy
metrics.brier_score_loss(y_true, y_prob[, …])	Compute the Brier score.
metrics.classification_report(y_true, y_pred)	Build a text report showing the main classification metrics
metrics.cohen_kappa_score(y1, y2[, labels, …])	Cohen’s kappa: a statistic that measures inter-annotator agreement.
metrics.confusion_matrix(y_true, y_pred[, …])	Compute confusion matrix to evaluate the accuracy of a classification
metrics.f1_score(y_true, y_pred[, labels, …])	Compute the F1 score, also known as balanced F-score or F-measure
metrics.fbeta_score(y_true, y_pred, beta[, …])	Compute the F-beta score
metrics.hamming_loss(y_true, y_pred[, …])	Compute the average Hamming loss.
metrics.hinge_loss(y_true, pred_decision[, …])	Average hinge loss (non-regularized)
metrics.jaccard_similarity_score(y_true, y_pred)	Jaccard similarity coefficient score
metrics.log_loss(y_true, y_pred[, eps, …])	Log loss, aka logistic loss or cross-entropy loss.
metrics.matthews_corrcoef(y_true, y_pred[, …])	Compute the Matthews correlation coefficient (MCC)
metrics.precision_recall_curve(y_true, …)	Compute precision-recall pairs for different probability thresholds
metrics.precision_recall_fscore_support(…)	Compute precision, recall, F-measure and support for each class
metrics.precision_score(y_true, y_pred[, …])	Compute the precision
metrics.recall_score(y_true, y_pred[, …])	Compute the recall
metrics.roc_auc_score(y_true, y_score[, …])	Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
metrics.roc_curve(y_true, y_score[, …])	Compute Receiver operating characteristic (ROC)
metrics.zero_one_loss(y_true, y_pred[, …])	Zero-one classification loss.
'''

In [28]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score



In [32]:
metrics = [roc_auc_score, accuracy_score, classification_report, cohen_kappa_score, f1_score, recall_score, precision_score]

In [33]:
[print(metrics[i](y_pred, y_test)) for i in range(len(metrics))]

0.6038564840479032
0.6668692679220709
             precision    recall  f1-score   support

      False       0.88      0.70      0.78     77504
       True       0.24      0.51      0.33     14631

avg / total       0.78      0.67      0.71     92135

0.1427948449615677
0.3278071001511137
0.5115166427448569
0.241185949081534


[None, None, None, None, None, None, None]

here we could see that auc/kappa/f1 score could reveal the performance of model, while the other metrics are not valid enough. so we will choose these 3 metrics.

In [34]:
used_metrics = [roc_auc_score, cohen_kappa_score, f1_score]

### 3. final_fitting

In [35]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [36]:
data1 = h2o.import_file('transformed_data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
cols = data1.col_names

In [38]:
cols

['C1',
 'Lat',
 'Alt',
 'Provider',
 'FixSatCount',
 'HasRadialAccuracy',
 'HasVerticalAccuracy',
 'HasSpeed',
 'Speed',
 'HasBearing',
 'Bearing',
 'BearingAccuracy',
 'n',
 'Attack']

In [40]:
#split the data as described above
train, valid, test = data1.split_frame([0.7, 0.15], seed=1234)

#Prepare predictors and response columns
X_train = cols[:-1]     #last column is Attack, our desired response variable 
y_train = cols[-1]    

In [41]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [42]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [43]:
rf_fit1.train(X_train, y_train, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [44]:
rf_perf1 = rf_fit1.model_performance(test)

In [45]:
# Retreive test set auc
print(rf_perf1.auc())
print(rf_perf1.confusion_matrix())

0.9999999912112737
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5519436418265105: 


,False,True,Error,Rate
False,36455.0,0.0,0.0,(0.0/36455.0)
True,2.0,18725.0,0.0001,(2.0/18727.0)
Total,36457.0,18725.0,0.0,(2.0/55182.0)


In [46]:
# try more complicated model
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit1', ntrees = 200, seed=1)

In [47]:
rf_fit2.train(X_train, y_train, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [48]:
rf_perf2 = rf_fit2.model_performance(test)

In [49]:
# Retreive test set mse
print(rf_perf2.auc())
print(rf_perf2.confusion_matrix())

0.9999999729014275
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4648073147679679: 


,False,True,Error,Rate
False,36453.0,2.0,0.0001,(2.0/36455.0)
True,0.0,18727.0,0.0,(0.0/18727.0)
Total,36453.0,18729.0,0.0,(2.0/55182.0)


This is the final step of whole project. During this project we discussed how to abstract the problem to mathmatical problem and solve it with machine learning algorithms gradually. A lot of data processing are discussed.